In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import tensorflow as tf
import numpy as np
import PIL
#import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Sequential
from tensorflow.compat.v1 import ConfigProto

model_path = "/media/2/Network/pretrained_model/vgg_model.h5"
#model_path = "/media/2/Network/pretrained_model/back_layers.h5"
#img_path = "/media/2/Network/Imagenet_dup/val/n02074367"# dugong
data_path = "/media/2/Network/Imagenet_dup/"
feature4_path = "/media/2/Network/extracted_feature/whole_not_shuffle_to_15"

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
config.gpu_options.allow_growth = True
tf.__version__

'2.3.0'

In [7]:
model=None
def ready_model(model_path,layer):
    model=load_model(model_path) # whole model
    # pooling 4
    til_pooling4_model=None
    til_pooling4_predict = None
    til_pooling5_model = None
    til_pooling5_predict = None
    if layer in 'pooling4':
        print('pooling4 session')
        #til_pooling4_model = Sequential([layer for layer in model.layers[:15]]) 
        #til_pooling4_model.build((None, 224,224,3))
        til_pooling4_predict = Sequential([layer for layer in model.layers[15:]]) 
        til_pooling4_predict.build((None, 14,14,512))
    # pooling5
    elif layer in 'pooling5':
        print('pooling5 session')
        til_pooling5_model = Sequential([layer for layer in model.layers[:19]]) 
        til_pooling5_model.build((None, 224,224,3))
        til_pooling5_predict = Sequential([layer for layer in model.layers[19:]]) 
        til_pooling5_predict.build((None, 7,7,512))
    return til_pooling4_model,til_pooling4_predict, til_pooling5_model,til_pooling5_predict
# load model & split pooling4 & pooling5
#til_pooling4_model=None
til_pooling4_predict=None
'''
til_pooling5_model=None
til_pooling5_predict=None
'''
til_pooling4_model,til_pooling4_predict, til_pooling5_model,til_pooling5_predict = ready_model(model_path,'pooling4')

pooling4 session


In [8]:
# load original image
#data_path = "/media/2/Network/Imagenet_dup/"
def load_original_dataset(data_path):
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True,
                                                               validation_split=0.2)
    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    training_set = train_datagen.flow_from_directory(data_path+"train", target_size=(224,224),
                                                batch_size =32,subset='training')
    val_set = train_datagen.flow_from_directory(data_path+"train",target_size=(224,224),
                                         batch_size=32, subset='validation')
                                         
                                         
    test_set = val_datagen.flow_from_directory(data_path+"val",target_size=(224,224),
                                         batch_size=32)
    print(training_set[0][0][0].shape)
    return training_set,val_set,test_set

In [16]:
feature_list =  os.listdir(feature4_path)
feature_list = sorted(feature_list)
len(feature_list),feature4_path

(87, '/media/2/Network/extracted_feature/whole_not_shuffle_to_15')

In [17]:
!ls /media/2/Network/extracted_feature/whole_not_shuffle_to_15/seq_16_pkt_error
error_feature_path = "/media/2/Network/extracted_feature/whole_not_shuffle_to_15/seq_16_pkt_error"

testing_features_0.npy	  training_features_24.npy  training_features_53.npy
testing_features_1.npy	  training_features_25.npy  training_features_54.npy
testing_features_2.npy	  training_features_26.npy  training_features_55.npy
testing_features_3.npy	  training_features_27.npy  training_features_56.npy
testing_label.npy	  training_features_28.npy  training_features_57.npy
training_features_00.npy  training_features_29.npy  training_features_58.npy
training_features_01.npy  training_features_30.npy  training_features_59.npy
training_features_02.npy  training_features_31.npy  training_features_60.npy
training_features_03.npy  training_features_32.npy  training_features_61.npy
training_features_04.npy  training_features_33.npy  training_features_62.npy
training_features_05.npy  training_features_34.npy  training_label.npy
training_features_06.npy  training_features_35.npy  validation_features_00.npy
training_features_07.npy  training_features_36.npy  validation_features_01.npy
training_fe

In [13]:


# add error in feature -> deprecated since error feature is store in disk
num_of_error = 16
def error_injection(feature,num_of_error):
    num_ch = feature.shape[3]
    for i, data in enumerate(feature):
        #print(i,data.shape)
        start = (i*num_of_error) % num_ch
        end = ((i+1)*num_of_error) % num_ch
        data[:,:,start:end] = 0
# classify feature
def error_feature_store(feature_list,num_of_error,feature4_path,error_feature_path):
    test_feature = []
    test_label = []
    train_feature = []
    train_label =[]
    val_feature = []
    val_label =[]
    for data in feature_list:
    #print(feature4_path+"/"+data)
        if 'train' in data:
            if 'feature' in data:
                tmp = np.load(feature4_path+"/"+data,mmap_mode='c')
                error_injection(tmp,num_of_error)
                np.save(error_feature_path+data,tmp)
                tmp = None
            else :
                tmp = np.load(feature4_path+"/"+data,mmap_mode='c')
                np.save(error_feature_path+data,tmp)
                tmp = None
       #print("train in",data)
        elif 'test' in data:
        #print("test in",data)
            if 'feature' in data:
                tmp = np.load(feature4_path+"/"+data,mmap_mode='c')
                error_injection(tmp,num_of_error)
                np.save(error_feature_path+data,tmp)
                tmp = None
            else:
                tmp = np.load(feature4_path+"/"+data,mmap_mode='c')
                np.save(error_feature_path+data,tmp)
                tmp = None
        elif 'validation' in data:
        #print("val in",data)
            if 'feature' in data:
                tmp = np.load(feature4_path+"/"+data,mmap_mode='c')
                error_injection(tmp,num_of_error)
                np.save(error_feature_path+data,tmp)
                tmp = None
            else:
                tmp = np.load(feature4_path+"/"+data,mmap_mode='c')
                np.save(error_feature_path+data,tmp)
            tmp = None
        gc.collect()
    
    print("error injection & store end")
# ex error_feature_store(feature_list,num_of_error,feature4_path,error_feature_path)

In [18]:
feature_list =  os.listdir(error_feature_path)
feature_list = sorted(feature_list)
len(feature_list)

86

In [8]:
# load feature from disk
# classify feature
test_feature = []
test_label = []
train_feature = []
train_label =[]
val_feature = []
val_label =[]
for data in feature_list:
    #print(feature4_path+"/"+data)
    if 'train' in data:
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            train_feature.append(tmp)
        else :
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            train_label.append(tmp)
       #print("train in",data)
    elif 'test' in data:
        #print("test in",data)
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            test_feature.append(tmp)
        else:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            test_label.append(tmp)
    elif 'validation' in data:
        #print("val in",data)
        if 'feature' in data:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            val_feature.append(tmp)
        else:
            tmp = np.load(error_feature_path+"/"+data,mmap_mode='c')
            val_label.append(tmp)

In [14]:
feature4_path

'/media/2/Network/extracted_feature/whole_not_shuffle_to_15'

In [10]:
back_layer = tf.keras.Sequential()
back_layer.add(tf.keras.layers.Flatten(name='flatten1'))
fc_layer = tf.keras.layers.Dense(14*14*512,input_dim=14*14*512,activation='tanh')
back_layer.add(fc_layer)
back_layer.add(tf.keras.layers.Reshape((14,14,512)))
for layer in til_pooling4_predict.layers[:]: # til_pooling4_predict.layers[1:]
    layer.trainable = False
    back_layer.add(layer)

#back_layer = tf.keras.layers.Concatenate()(fc_layer,til_pooling4_predict)
# model  compile
def scheduler(epoch,lr):
    if epoch < 15:
        return lr
    else:
        return lr*tf.math.exp(-0.1)
MODEL_SAVE_FOLDER_PATH = './models'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_loss:.4f}.hdf5'
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=model_path, 
                               monitor='val_loss', verbose=1,
                               save_best_only=True)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
optimizer = tf.keras.optimizers.Adam(lr=1e-4, decay=0)
#til_pooling4_predict.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
back_layer.build((None,14,14,512))
back_layer.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

back_layer.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100352)            1007062425
_________________________________________________________________
reshape_1 (Reshape)          (None, 14, 14, 512)       0         
_________________________________________________________________
block5_conv1 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        (None, 14, 14, 512)       2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   (None, 7, 7, 512)        

In [ ]:
%%capture output
max_epoch = 30
feature_len = len(val_feature[0])
val_feature_len = len(val_feature)
for epoch in range(1,max_epoch+1):
    print("epoch :",epoch)
    for i, feature in enumerate(train_feature):
        print(feature.shape,val_feature[i].shape)
        start = i*feature_len
        end  = (i+1)*feature_len
        y = back_layer.fit(feature,train_label[0][start:end],
                                 batch_size=32,epochs=1,
                                 validation_data=(val_feature[i%val_feature_len],val_label[0][start:end]),
                                   callbacks=[cb_checkpoint],verbose=1,use_multiprocessing=False)
    

In [47]:
len(val_feature)

16

In [ ]:
class F4F(tf.keras.layers.Layer):
    def __init__(self, num_ouputs):
        super(F4F, self).__init__()
        self.num_outputs = num_outputs
    def build(self, input_shape):
        self.kernel = self.add_variable()

In [ ]:
# 중간 단계에서 